# CIS 545 Homework 2: Graphs

In [1]:
# Execute this once, the first time you run
!pip install networkx

# Disable Python warning messages - you should probably only run this before submission

import warnings
warnings.filterwarnings('ignore')

You are using pip version 9.0.1, however version 9.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


## Step 2.1 Spark Setup

In [2]:
# TODO: Connect to Spark as per Step 2.1
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

spark = SparkSession.builder.appName('Graphs-HW2').getOrCreate()

In [3]:
# Load some dummy data, which should be overwritten in Step 2.2

answers_sdf = spark.createDataFrame([{'from_node': 123, 'to_node': 456},\
                                    {'from_node': 456, 'to_node': 789},
                                    {'from_node': 456, 'to_node': 890}])
comments_answers_sdf = spark.createDataFrame([{'from_node': 123, 'to_node': 456}])
comments_questions_sdf = spark.createDataFrame([{'from_node': 123, 'to_node': 456}])

graph_sdf = spark.createDataFrame([{'from_node': 123, 'to_node': 456}])

answers_sdf.show()
comments_answers_sdf.show()
comments_questions_sdf.show()
graph_sdf.show()

+---------+-------+
|from_node|to_node|
+---------+-------+
|      123|    456|
|      456|    789|
|      456|    890|
+---------+-------+

+---------+-------+
|from_node|to_node|
+---------+-------+
|      123|    456|
+---------+-------+

+---------+-------+
|from_node|to_node|
+---------+-------+
|      123|    456|
+---------+-------+

+---------+-------+
|from_node|to_node|
+---------+-------+
|      123|    456|
+---------+-------+



## Step 2.2 Loading

In [4]:
# TODO: load data as per Step 2.2

answers_sdf = spark.read.load('sx-stackoverflow-a2q.txt', format = "text")
comments_answers_sdf = spark.read.load('sx-stackoverflow-c2a.txt', format = "text")
comments_questions_sdf = spark.read.load('sx-stackoverflow-c2q.txt', format = "text")

answers_sdf.show()
comments_answers_sdf.show()
comments_questions_sdf.show()

+----------------+
|           value|
+----------------+
|  9 8 1217567877|
|  1 1 1217573801|
| 13 1 1217606247|
| 17 1 1217617639|
| 48 2 1217618182|
| 17 1 1217618239|
| 19 9 1217618357|
|13 23 1217618560|
|13 11 1217618799|
|23 23 1217619360|
|35 33 1217620542|
|39 33 1217620597|
|43 22 1217620971|
|17 32 1217621272|
|39 40 1217621416|
|37 40 1217621670|
|45 45 1217621917|
|17 17 1217622124|
|49 13 1217623079|
|13 23 1217623216|
+----------------+
only showing top 20 rows

+--------------------+
|               value|
+--------------------+
|     1 91 1220713630|
|     3 91 1220713792|
|  380 350 1220715736|
|4642 2257 1220734307|
|4642 1324220 1220...|
|2495 4285 1220736640|
|4642 4893 1220737355|
|2515 4903 1220738560|
|2515 4893 1220739071|
|  199 199 1220741079|
|  658 658 1220742035|
| 292 1694 1220744791|
|4213 4213 1220747610|
|4213 1694 1220747855|
|2353001 825 12207...|
|4064 3666 1220754304|
| 4064 828 1220755251|
|4906 3205 1220757270|
|3394 1569 1220757715|
|4906 4903 1

## Step 2.2 Results

In [5]:
answers_sdf.count()

17823525

In [6]:
answers_sdf.show()

+----------------+
|           value|
+----------------+
|  9 8 1217567877|
|  1 1 1217573801|
| 13 1 1217606247|
| 17 1 1217617639|
| 48 2 1217618182|
| 17 1 1217618239|
| 19 9 1217618357|
|13 23 1217618560|
|13 11 1217618799|
|23 23 1217619360|
|35 33 1217620542|
|39 33 1217620597|
|43 22 1217620971|
|17 32 1217621272|
|39 40 1217621416|
|37 40 1217621670|
|45 45 1217621917|
|17 17 1217622124|
|49 13 1217623079|
|13 23 1217623216|
+----------------+
only showing top 20 rows



In [7]:
answers_sdf.printSchema()

root
 |-- value: string (nullable = true)



In [8]:
comments_answers_sdf.count()

25405374

In [9]:
comments_answers_sdf.show(10)

+--------------------+
|               value|
+--------------------+
|     1 91 1220713630|
|     3 91 1220713792|
|  380 350 1220715736|
|4642 2257 1220734307|
|4642 1324220 1220...|
|2495 4285 1220736640|
|4642 4893 1220737355|
|2515 4903 1220738560|
|2515 4893 1220739071|
|  199 199 1220741079|
+--------------------+
only showing top 10 rows



In [10]:
comments_answers_sdf.printSchema()

root
 |-- value: string (nullable = true)



In [11]:
comments_questions_sdf.count()

20268151

In [12]:
comments_questions_sdf.show(10)

+--------------------+
|               value|
+--------------------+
|4550 4550 1220729190|
|  242 184 1220733503|
|4213 4946 1220768149|
|    91 91 1220768295|
|2658 1874 1220771891|
|4035 1874 1220773037|
|2257 4489 1220802041|
|  577 577 1220834891|
|4489 4489 1220853536|
| 828 2783 1220854143|
+--------------------+
only showing top 10 rows



In [13]:
comments_questions_sdf.printSchema()

root
 |-- value: string (nullable = true)



## Step 2.3

In [14]:
# TODO: wrangling work in Step 2.3.  Add as many Cells as you need

answers_sdf.createOrReplaceTempView('answers')
answers_sdf = spark.sql('select split (value, " ") as from_node, split (value, " ") as to_node, split (value, " ") as edge_type from answers')

answers_sdf = answers_sdf.select(F.split(answers_sdf.value, " ")[0].alias("from_node").cast("int"),\
                  F.split(answers_sdf.value, " ")[1].alias("to_node").cast("int"),\
                  F.split(answers_sdf.value, " ")[2].alias("edge_type").cast("string"))
answers_sdf = answers_sdf.withColumn('edge_type', F.lit('answers'))

comments_answers_sdf = comments_answers_sdf.select(F.split(comments_answers_sdf.value, " ")[0].alias("from_node").cast("int"),\
                  F.split(comments_answers_sdf.value, " ")[1].alias("to_node").cast("int"),\
                  F.split(comments_answers_sdf.value, " ")[2].alias("edge_type").cast("string"))
comments_answers_sdf = comments_answers_sdf.withColumn('edge_type', F.lit('comment-on-answer'))

comments_questions_sdf = comments_questions_sdf.select(F.split(comments_questions_sdf.value, " ")[0].alias("from_node").cast("int"),\
                  F.split(comments_questions_sdf.value, " ")[1].alias("to_node").cast("int"),\
                  F.split(comments_questions_sdf.value, " ")[2].alias("edge_type").cast("string"))
comments_questions_sdf = comments_questions_sdf.withColumn('edge_type', F.lit('comment-on-question'))

answers_sdf.show()
comments_answers_sdf.show()
comments_questions_sdf.show()

AttributeError: 'DataFrame' object has no attribute 'value'

In [ ]:
graph_sdf = answers_sdf.unionAll(comments_answers_sdf).unionAll(comments_questions_sdf)
graph_sdf.show()

## Step 2.3 Results

In [ ]:
answers_sdf.count()

In [ ]:
answers_sdf.show(5)

In [ ]:
answers_sdf.printSchema()

In [ ]:
comments_answers_sdf.count()

In [ ]:
comments_answers_sdf.show(5)

In [ ]:
comments_answers_sdf.printSchema()

In [ ]:
 comments_questions_sdf.count()

In [ ]:
comments_questions_sdf.show(5)

In [ ]:
comments_questions_sdf.printSchema()

In [ ]:
graph_sdf.count()

In [ ]:
graph_sdf.show(5)

In [ ]:
graph_sdf.printSchema()

## Step 2.4

In [ ]:
# You may put any computations you need here

## Step 2.4.1 Results

In [ ]:
# TODO: output dataframe with top 10 users by number of questions
answers_sdf_count = answers_sdf.groupby(answers_sdf['to_node']).count()

answers_sdf_count = answers_sdf_count.createOrReplaceTempView("answers_sdf_count_view")
number_of_questions = spark.sql('select * from answers_sdf_count_view order by count DESC')

number_of_questions = number_of_questions.createOrReplaceTempView("number_of_questions_view")
number_of_questions = spark.sql('select to_node as user, count as ansCounts from number_of_questions_view')

number_of_questions.show(10)

In [ ]:
# TODO: output top 10 users by number of answers to questions by distinct users
answers_sdf_count = answers_sdf.groupby(answers_sdf['from_node']).count()

answers_sdf_count = answers_sdf_count.createOrReplaceTempView("answers_sdf_count_view")
number_of_answers_to_questions = spark.sql('select * from answers_sdf_count_view order by count DESC')

number_of_answers_to_questions = number_of_answers_to_questions.createOrReplaceTempView("number_of_answers_to_questions_view")
number_of_answers_to_questions = spark.sql('select from_node as user, count as ansCounts from number_of_answers_to_questions_view')

number_of_answers_to_questions.show(10)

## Step 2.4.2 Results

In [ ]:
# TODO: number of users whose questions have never been answered but commented on

left = comments_questions_sdf
right = answers_sdf

combine = left.join(right, left.to_node == right.to_node, how = 'leftanti').drop_duplicates(['to_node'])
combine.count()
combine.show(5)

## Step 2.4.3 Results

In [ ]:
# TODO: top 10 pairs of users by mutual answers, along with the number of questions they have mutually answered
pairs_sdf = answers_sdf.groupby(answers_sdf['from_node'], answers_sdf['to_node']).count()
pairs_sdf.createOrReplaceTempView("pairs_sdf_view")
pairs_sdf_ordered = spark.sql('select * from pairs_sdf_view order by count DESC')

pairs_sdf_ordered.createOrReplaceTempView("pairs_sdf_ordered_view")
pairs_sdf_ordered = spark.sql('\
select * from pairs_sdf_ordered_view \
where pairs_sdf_ordered_view.from_node != pairs_sdf_ordered_view.to_node')
# pairs_sdf_ordered.show()

pairs_sdf_ordered.createOrReplaceTempView("pairs_sdf_ordered_view")
pairs_entire_sdf = spark.sql('\
SELECT po1.from_node AS user1, po1.to_node AS user2, po1.count + po2.count AS ansCounts \
FROM pairs_sdf_ordered_view AS po1 \
INNER JOIN pairs_sdf_ordered_view AS po2 \
ON po1.from_node = po2.to_node AND po1.to_node = po2.from_node \
WHERE po1.from_node < po1.to_node \
ORDER BY ansCounts DESC ')
pairs_entire_sdf.show()

# Step 3

In [ ]:
# TODO: Fill in according to HW spec

graph_sdf_out_count = graph_sdf.groupby(graph_sdf['from_node']).count()
graph_sdf_out_count = graph_sdf_out_count.createOrReplaceTempView("graph_sdf_out_count_view")
highest_outdegree_sdf = spark.sql('select * from graph_sdf_out_count_view order by count DESC')

graph_sdf_in_count = graph_sdf.groupby(graph_sdf['to_node']).count()
graph_sdf_in_count = graph_sdf_in_count.createOrReplaceTempView("graph_sdf_in_count_view")
highest_indegree_sdf = spark.sql('select * from graph_sdf_in_count_view order by count DESC')

## Step 3 Results

In [ ]:
highest_indegree_sdf.show(5)

In [ ]:
highest_outdegree_sdf.show(5)

## Step 4

In [ ]:
# TODO: insert code as you like

In [ ]:
schema = StructType([StructField("node",IntegerType(),True)])
schema1 = StructType([StructField("node",IntegerType(),True),StructField("depth",IntegerType(),True)])

def spark_bfs(G, origins, max_depth):
    
    ##Your logic goes here
    unexplored_sdf = G.repartition(100,'from_node').cache()
    frontier_sdf = spark.createDataFrame(origins, schema).cache()
    visited_sdf = spark.createDataFrame(origins, schema1)
    visited_sdf = visited_sdf.union(frontier_sdf.withColumn('depth',F.lit(0)))
    
    for depth in range(max_depth):
        
        unexplored_sdf = unexplored_sdf.join(frontier_sdf, frontier_sdf.node == unexplored_sdf.to_node,'leftanti').cache()
        edge_sdf = unexplored_sdf.join(frontier_sdf, frontier_sdf.node == unexplored_sdf.from_node,'inner')
        next_sdf = edge_sdf.select(edge_sdf.to_node.alias('node'))
        visited_sdf = visited_sdf.union(next_sdf.withColumn('depth',F.lit(depth + 1)))
        frontier_sdf = next_sdf.drop_duplicates().cache()
    
    unexplored_sdf.unpersist()
    frontier_sdf.unpersist()
    return visited_sdf

## Step 4.1

In [ ]:
# TODO: comment out this line once your code is ready
bfs_sdf = spark.createDataFrame([{'node': 123, 'depth': 1}, {'node': 456, 'depth': 2}])

# TODO: enable this once your code is ready
origin_map = [{'node': 4550}, {'node': 242}]
bfs_sdf = spark_bfs(comments_questions_sdf, origin_map, 2)
bfs_sdf.count()

In [ ]:
# TODO: insert code as you like

## Step 4.1 Results

In [ ]:
bfs_sdf.show(10)

## Step 4.2

In [ ]:
#
# Step 4.2 Pre-processing
#
def friend_rec(input_sdf, graph_sdf):
    
    ## Your logic goes in here
    graph_sdf.cache()
    
    total_sdf = input_sdf.select('node')
    friends_out_sdf = total_sdf.join(graph_sdf, total_sdf.node == graph_sdf.from_node, 'inner')
    friends_out_sdf = friends_out_sdf.select(friends_out_sdf.node, friends_out_sdf.to_node.alias('friend'))
    
    friends_in_sdf = total_sdf.join(graph_sdf, total_sdf.node == graph_sdf.to_node, 'inner')
    friends_in_sdf = friends_in_sdf.select(friends_in_sdf.node, friends_in_sdf.from_node.alias('friend'))
    
    graph_sdf.unpersist()
    total_friends_sdf = friends_out_sdf.union(friends_in_sdf).distinct()
    total_friends_sdf = total_friends_sdf.repartition(100, 'node').cache()
    total_friends_sdf = total_friends_sdf.where(total_friends_sdf.node != total_friends_sdf.friend).cache()
    
    total_friends_sdf.createOrReplaceTempView('total_friends_view')
    friend_rec_sdf = spark.sql('select frec1.node as from_node, frec2.node as to_node, \
    count(frec1.friend) as connections \
    from total_friends_view as frec1 \
    inner join total_friends_view as frec2 \
    on frec1.friend = frec2.friend \
    where frec1.node != frec2.node \
    group by frec1.node, frec2.node').cache()
    
    friend_rec_sdf = friend_rec_sdf.where(friend_rec_sdf.connections >= 2).select('from_node', 'to_node').cache()
    friend_rec_sdf = friend_rec_sdf.join(total_friends_sdf, [total_friends_sdf.node == friend_rec_sdf.from_node, total_friends_sdf.friend == friend_rec_sdf.to_node], 'leftanti').drop_duplicates()
    
    total_friends_sdf.unpersist()
    return friend_rec_sdf

In [ ]:
filtered_bfs_sdf = bfs_sdf[bfs_sdf.depth == 2].groupBy('node', 'depth').count()
# count is a method
filtered_bfs_sdf = filtered_bfs_sdf.withColumnRenamed('count', 'num')
filtered_bfs_sdf = filtered_bfs_sdf[filtered_bfs_sdf.num > 1]
filtered_bfs_sdf = filtered_bfs_sdf.drop('num')
filtered_bfs_sdf

In [ ]:
# TODO: enable this when your function works

friend_recommendations_sdf = friend_rec(filtered_bfs_sdf, comments_questions_sdf)
friend_recommendations_sdf.count()

## Step 4.2 Results

In [ ]:
friend_recommendations_sdf.count()

In [ ]:
friend_recommendations_sdf.show()

## Step 4.3: Graph visualization

### Once you have excecuted the cells in Step 4.2 and you have friend_recommendations_sdf, lets create friend_recommendations_df using toPandas(). This creates an in-memory dataFrame that we can use to build the graph. Here we have used ('from_node','to_node') as column names in friend_recommendations_sdf, please change it to what you have used in yours.

In [ ]:
import networkx as nx

# TODO: create friend_graph NetworkX graph from friend_recommendations_df from friend_recommendations_sdf
friend_rec_df = friend_recommendations_sdf.toPandas()
friend_graph = nx.from_pandas_dataframe(friend_rec_df, 'from_node','to_node')

## Step 4.3 Results

In [ ]:
print ("Number of nodes (characters) in this graph is", friend_graph.order()) # number of nodes
print ("Number of edges in this graph is", len(friend_graph.edges())) # number of edges
print ("Graph diameter is", nx.diameter(friend_graph)) # maximum eccentricity

In [ ]:
 %matplotlib inline
 nx.draw(friend_graph)

In [ ]:
new_sdf = answers_sdf.groupby(['from_node']).count()
answers_sdf.show()